In [1]:
# Install Azure Custom Vision Python SDK

!pip install azure-cognitiveservices-vision-customvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 10.7 MB/s eta 0:00:00


In [2]:
# Set Timezone. Thought this might be needed, but it dosen't seem to make any difference.

import pytz
from datetime import datetime

# Convert naive datetime to aware datetime using pytz
naive_datetime = datetime.now()
utc_datetime = naive_datetime.replace(tzinfo=pytz.UTC)

In [85]:
# Configure Azure Custom Vision Enpoints and Training and Prediciton Keys

import os
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

#ENDPOINT = "https://mrlevel05fsdmission01customvision.cognitiveservices.azure.com/"
#TRAINING_KEY = "5wJijZ5a9BOxcIYNgFkXofWtfjGhYmHYDdYkrtHnUWhncb5JiGJnJQQJ99AKACL93NaXJ3w3AAAJACOGXJC4"
#PREDICTION_KEY = "6lQavznVjoPDfpVneIl74sAyJZmz4kSIbH7gpvPbpPdgA6SIpe13JQQJ99AKACL93NaXJ3w3AAAIACOGymPJ"
#PREDICTION_RESOURCE_ID = "/subscriptions/27dfa5db-72e7-41cc-8e68-be3fd4e7ad01/resourceGroups/mr-level-05-fsd-mission-01-custom-vision/providers/Microsoft.CognitiveServices/accounts/mrlevel05fsdmission01customvision-Prediction"
#PREDICTION_ENDPOINT = "https://mrlevel05fsdmission01customvision-prediction.cognitiveservices.azure.com"

ENDPOINT = "https://mrlevel05fsdmission01customvision.cognitiveservices.azure.com/"
TRAINING_KEY = "5wJijZ5a9BOxcIYNgFkXofWtfjGhYmHYDdYkrtHnUWhncb5JiGJnJQQJ99AKACL93NaXJ3w3AAAJACOGXJC4"

PREDICTION_KEY = "6T2z43S7OhdkkQSb245zqD727OhfhPd8ZMtrGugfEqGYsxV6rXaUJQQJ99AKACL93NaXJ3w3AAAIACOG24EF"
PREDICTION_RESOURCE_ID = "/subscriptions/27dfa5db-72e7-41cc-8e68-be3fd4e7ad01/resourceGroups/mr-level-05-fsd-mission-01-custom-vision/providers/Microsoft.CognitiveServices/accounts/mrlevel05fsdmission01customvision-Prediction"
PREDICTION_ENDPOINT = "https://mrlevel05fsdmission01customvision-prediction.cognitiveservices.azure.com/"


In [74]:
# Authenticate with the Azure Custom Vision Training Client

credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

In [75]:
# Check the connection by listing all projects

try:
    projects = trainer.get_projects()
    print("Connected to Custom Vision. Here are your projects:")
    for project in projects:
        print(f" - {project.name} (ID: {project.id})")
except Exception as e:
    print("Failed to connect to Custom Vision. Please check your credentials and endpoint.")
    print("Error:", e)

Connected to Custom Vision. Here are your projects:
 - Vehicle Class Identification 02 (ID: 396ef7f4-2ec0-4bd5-990d-af98116abfbe)
 - Vehicle Class Identification (ID: 11b2df83-5803-4460-a114-7ab4050aacfb)


In [76]:
# Find and use existing project by name

project_name = "Vehicle Class Identification 02"
project = None

for proj in projects:
    if proj.name == project_name:
        project = proj
        break

if project:
    print(f"Using existing project: {project.name} (ID: {project.id})")
else:
    print(f"Project '{project_name}' not found. Please check the project name.")

Using existing project: Vehicle Class Identification 02 (ID: 396ef7f4-2ec0-4bd5-990d-af98116abfbe)


In [9]:
# Install GDown for downloading data from Google Drive

!pip -q install gdown

In [10]:
# Download the ZIP archive of the dataset

import gdown

share_link = "https://drive.google.com/file/d/1o8ZxFqylNY37aoDljaFLhQDxv_iu9PdI/view?usp=drive_link"
file_id = share_link.split('/d/')[1].split('/view')[0]
gdown.download(f"https://drive.google.com/uc?id={file_id}", "ai-vehicle-id-dataset.zip", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1o8ZxFqylNY37aoDljaFLhQDxv_iu9PdI
From (redirected): https://drive.google.com/uc?id=1o8ZxFqylNY37aoDljaFLhQDxv_iu9PdI&confirm=t&uuid=ea56449c-6ded-45a4-9625-12bfe99bc278
To: /content/ai-vehicle-id-dataset.zip
100%|██████████| 2.06G/2.06G [00:50<00:00, 41.2MB/s]


'ai-vehicle-id-dataset.zip'

In [11]:
# Unpack the dataset into /content/train for training data and /content/val for validation data. Remove /content/__MACOSX folder.

import zipfile
import shutil
import os

print("Unpacking dataset into /content/")

with zipfile.ZipFile('/content/ai-vehicle-id-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

try:
  shutil.rmtree('/content/__MACOSX')
except FileNotFoundError:
  print("'/content/__MACOSX' not found, skipping removal.")

Unpacking dataset into /content/


In [12]:
# Display Dataset Description from /content/README.md

from IPython.display import Markdown

with open('/content/README.md', 'r') as f:
  md_content = f.read()

display(Markdown(md_content))

# AI Vehicle Identification Datasets

Dataset Folder Structure

```
/train
  /cab
  /convertible
  /coupe
  /hatchback
  /minivan
  /Negative
  /sedan
  /suv
  /truck
  /van
  /wagon
/val
  /cab
  /convertible
  /coupe
  /hatchback
  /minivan
  /Negative
  /sedan
  /suv
  /truck
  /van
  /wagon  
 
```

Dataset Contents

* Image counts in train folders:
	
	* sedan: 1907
	* van: 291
	* suv: 1437
	* Negative: 200
	* truck: 347
	* cab: 719
	* hatchback: 554
	* coupe: 1054
	* minivan: 250
	* convertible: 1036
	* wagon: 253
	
* Image counts in validation folders:
	
	* sedan: 1880
	* van: 287
	* suv: 1418
	* Negative: 79
	* truck: 240
	* cab: 711
	* hatchback: 549
	* coupe: 1042
	* minivan: 248
	* convertible: 1022
	* wagon: 250

The images were sourced from the following datasets:

* Car Images - Standford Cars Dataset
	* Image Files - Download via KaggleHub - Dataset Name: jutrera/stanford-car-dataset-by-classes-folder

		```
		import kagglehub
		
		# Download latest version
		path = kagglehub.dataset_download("jutrera/stanford-car-dataset-by-classes-folder")
		
		print("Path to dataset files:", path)
		```

	* Dataset Description [Standford Cars Dataset](https://www.kaggle.com/datasets/jutrera/stanford-car-dataset-by-classes-folder/data)
	* License [ImageNet License](https://www.image-net.org/download.php)
* Truck Images - Images.CV - Trucks Dataset
	* Image Files [Images.CV Trucks Dataset](https://images.cv/dataset/truck-image-classification-dataset) 
	* License [Various Licenses - No Specific License Specified](https://images.cv/datasets-licenses)
* Negative Images - BG-20K Datset of Backgrounds
	* Image Files [BG-20K Google Drive](https://drive.google.com/drive/folders/1ZBaMJxZtUNHIuGj8D8v3B9Adn8dbHwSS)
	* Dataset Description [BG-20K GitHub](https://github.com/JizhiziLi/GFM?tab=readme-ov-file#bg-20k)
	* Usage Agreement [Dataset Agreement](https://jizhizili.github.io/files/gfm_datasets_agreements/BG-20k_Dataset_Release_Agreement.pdf)
	* License [MIT License](https://github.com/JizhiziLi/GFM/blob/master/LICENSE)



In [ ]:
# Get Images

!pip install kagglehub

In [ ]:
train_path = path + "/car_data/car_data/train"

In [45]:
# Extract Tags Names from Folder Names

train_path = "/content/train"
tags_dataset = [folder for folder in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, folder))]
print(f"Found tags: {tags_dataset}")
print(len(tags_dataset))

Found tags: ['wagon', 'convertible', 'minivan', 'cab', 'Negative', 'van', 'truck', 'coupe', 'suv', 'sedan', 'hatchback']
11


In [46]:
# Display No. of Tags

print(f"No. of Tags: {len(tags_dataset)}")

No. of Tags: 11


In [77]:
# Display Tag Names

import pandas as pd

series = pd.Series(list(tags_dataset))
series

,0
0,wagon
1,convertible
2,minivan
3,cab
4,Negative
5,van
6,truck
7,coupe
8,suv
9,sedan


In [58]:
# @title Default title text
REMOVE_EXISTING_TAGS = True # @param {"type":"boolean","placeholder":"False"}
REMOVE_EXISTING_IMAGES = False # @param {"type":"boolean","placeholder":"False"}



In [59]:
REMOVE_EXISTING_TAGS, REMOVE_EXISTING_IMAGES

(True, True)

In [78]:
# Remove existing tags - This removes tags, but also moves images to the 'untagged' category

if REMOVE_EXISTING_TAGS:

  # List all tags in the project
  tags = trainer.get_tags(project.id)

  # Iterate over the tags and delete each one
  for tag in tags:
      try:
          trainer.delete_tag(project.id, tag.id)
          print(f"Tag '{tag.name}' deleted successfully.")
      except Exception as e:
          print(f"Failed to delete tag '{tag.name}': {e}")

In [61]:
# Remove existing images - This dosen't seem to delete untagged images.

if REMOVE_EXISTING_IMAGES:

  from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient

  # Step 1: Get all images in the project
  print("Retrieving all images in the project...")
  existing_images = trainer.get_images(project.id) #, tagging_status='Untagged')

  # Step 2: Delete all images using the correct method
  for image in existing_images:
      print(f"Deleting image {image.id}")  # Accessing the 'id' attribute instead of 'name'
      # Use delete_images() method to delete images
      trainer.delete_images(project.id, [image.id])

  print("All images have been deleted.")


Retrieving all images in the project...
Deleting image a848084b-098b-40ed-bafa-cec2d9077c76
Deleting image 0cd5ad2a-8e75-441f-b3d0-13e37f616280
Deleting image 6d74bd14-6002-4821-8d81-7ef1082bfe8f
Deleting image 8f357dde-783c-496f-8608-e8c7b6c08132
Deleting image fba02688-faf6-4ae4-bd93-607de71ff33c
Deleting image 43a5b8dd-326a-4bbd-8305-3a3b5a183b8a
Deleting image c3e00415-0bd8-4d5b-94a6-f3efd3e7794a
Deleting image 5a917178-f4f9-4993-b240-c214c68eaccc
Deleting image 09e74e2f-6a17-438a-b218-a82e20e9d718
Deleting image 0f0ce5e6-5317-4081-bdaa-ac5ae03fd433
Deleting image 664a0eab-8dd6-46c6-8d1e-06d9868b1e03
Deleting image 1d72f0fd-ed52-49d3-9712-6a8d2795c1f2
Deleting image fe501497-345f-4a51-9a82-35c1453c41ce
Deleting image 0b83bf2d-fd94-4126-a90d-4e77cd0b18d6
Deleting image 5fe3ada4-d831-4ff5-9bb8-d9cae9d496eb
Deleting image 67d5664c-a06b-4fc9-ba47-2349fc007a18
Deleting image 751e210e-d288-425b-9540-63e455a69984
Deleting image a352ec22-4857-4141-8579-0a570cd4c983
Deleting image 826824d7-

In [63]:
# Verify that all images have been deleted

if REMOVE_EXISTING_IMAGES:

  remaining_images = trainer.get_images(project.id)

  if not remaining_images:
      print("Verification complete: All images have been successfully deleted.")
  else:
      print(f"Images still present: {len(remaining_images)}")

Images still present: 50


In [79]:
# Verify all tags have been deleted

# List all tags in the project
tags_CV = trainer.get_tags(project.id)

# Iterate over the tags
for tag in tags_CV:
        print(f"Tag: '{tag.name}'")

In [80]:
# Enumerate the folders in /content/train and create a dictionary with folder name as the key, and folder path as the value

import os

def enumerate_folders(root_dir):
  """
  Enumerates folders within a given root directory and creates a dictionary
  with folder names as keys and folder paths as values.
  """
  folder_dict = {}
  for folder_name in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder_name)
    if os.path.isdir(folder_path):
      folder_dict[folder_name] = folder_path
  return folder_dict

# Example usage:
train_path = "/content/train"
tagged_folders = enumerate_folders(train_path)

tagged_folders

{'wagon': '/content/train/wagon',
 'convertible': '/content/train/convertible',
 'minivan': '/content/train/minivan',
 'cab': '/content/train/cab',
 'Negative': '/content/train/Negative',
 'van': '/content/train/van',
 'truck': '/content/train/truck',
 'coupe': '/content/train/coupe',
 'suv': '/content/train/suv',
 'sedan': '/content/train/sedan',
 'hatchback': '/content/train/hatchback'}

In [81]:
# Upload Images with Tags

import os
import random
import time
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, ImageFileCreateBatch

# Define how many images you want per tag
images_per_tag = 400

# Define the delay time between uploads (in seconds)
rate_limit_delay = 0.5  # 0.5 seconds delay between batches to respect the rate limit

# Iterate through each tag and its associated folders
for tag_name, folder in tagged_folders.items():
    # Create the tag in the project
    tag = trainer.create_tag(project.id, tag_name)
    tag_id = tag.id

    images = []

    # Collect all images from all folders corresponding to the tag
    images_in_folder = [
        os.path.join(folder, fname)
        for fname in os.listdir(folder)
        if fname.lower().endswith(('jpg', 'jpeg', 'png'))
    ]
    images.extend(images_in_folder)

    # Remove duplicates by converting the list to a set and back to a list
    images = list(set(images))

    # Now, limit the total number of images to `images_per_tag`
    if len(images) > images_per_tag:
        images = random.sample(images, images_per_tag)

    # Upload images in batches
    batch_size = 50  # You can choose a suitable batch size
    for i in range(0, len(images), batch_size):
        print(f"Uploading batch {i // batch_size + 1} of {len(images) // batch_size + 1} for tag '{tag_name}'...")
        batch = images[i:i + batch_size]
        image_entries = []

        for image_path in batch:
            with open(image_path, "rb") as image_file:
                image_data = image_file.read()
                image_entry = ImageFileCreateEntry(name=os.path.basename(image_path), contents=image_data, tag_ids=[tag_id])
                image_entries.append(image_entry)
                # time.sleep(rate_limit_delay)

        # Create a batch from the image entries
        image_batch = ImageFileCreateBatch(images=image_entries)

        # Upload the image batch using the trainer
        upload_result = trainer.create_images_from_files(project.id, image_batch)


        # Check if the batch upload was successful
        if not upload_result.is_batch_successful:
            print(f"Batch upload failed or incomplete for tag_id {tag_name}.")
        else:
            print(f"Batch uploaded successfully for tag_id {tag_name}.")

        # Add a delay between each batch upload to respect the API rate limit
        time.sleep(rate_limit_delay)

# Check the result
print("Image upload complete.")

Uploading batch 1 of 6 for tag 'wagon'...
Batch uploaded successfully for tag_id wagon.
Uploading batch 2 of 6 for tag 'wagon'...
Batch uploaded successfully for tag_id wagon.
Uploading batch 3 of 6 for tag 'wagon'...
Batch uploaded successfully for tag_id wagon.
Uploading batch 4 of 6 for tag 'wagon'...
Batch uploaded successfully for tag_id wagon.
Uploading batch 5 of 6 for tag 'wagon'...
Batch upload failed or incomplete for tag_id wagon.
Uploading batch 6 of 6 for tag 'wagon'...
Batch uploaded successfully for tag_id wagon.
Uploading batch 1 of 9 for tag 'convertible'...
Batch uploaded successfully for tag_id convertible.
Uploading batch 2 of 9 for tag 'convertible'...
Batch uploaded successfully for tag_id convertible.
Uploading batch 3 of 9 for tag 'convertible'...
Batch uploaded successfully for tag_id convertible.
Uploading batch 4 of 9 for tag 'convertible'...
Batch uploaded successfully for tag_id convertible.
Uploading batch 5 of 9 for tag 'convertible'...
Batch uploaded suc

In [82]:
# Count Images per Tag - This gives incorrect images/tag - It's like it is an old count

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient

# Get project tags
tags = trainer.get_tags(project.id)

# Initialize a dictionary to track the number of images for each tag
tag_image_counts = {}

# Loop through the tags
for tag in tags:
    tag_name = tag.name
    tag_id = tag.id

    # Initialize a counter for the number of images
    image_count = 0

    # Get images for the tag (with top=1000 for large batches)
    results = trainer.get_images(
        project.id,
        tag_ids=[tag_id],
        tagging_status="Tagged",
        top=4000  # Adjust the value for your needs
    )

    # Count the number of images retrieved
    image_count = len(results)

    # Save the count for this tag
    tag_image_counts[tag_name] = image_count

# Print out the image count for each tag
print("Image count per tag:")
for tag_name, count in tag_image_counts.items():
    print(f"Tag '{tag_name}': {count} images")


Image count per tag:
Tag 'truck': 50 images
Tag 'suv': 50 images
Tag 'van': 50 images
Tag 'minivan': 50 images
Tag 'coupe': 50 images
Tag 'sedan': 50 images
Tag 'hatchback': 50 images
Tag 'cab': 50 images
Tag 'wagon': 50 images
Tag 'convertible': 50 images
Tag 'Negative': 50 images


In [83]:
# Begin the Training Process

import time

# Start the timer
start_time = time.time()

print("Starting training...")
iteration = trainer.train_project(project.id)

# Wait for training to complete
while iteration.status != "Completed":
    iteration = trainer.get_iteration(project.id, iteration.id)
    print(f"Training status: {iteration.status}")
    time.sleep(10)  # Wait for a few seconds before checking again

print("Training complete!")

# End the timer
end_time = time.time()

# Calculate elapsed time in seconds
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time:.2f} seconds")

# Optionally, publish the iteration to make it available for predictions
publish_iteration_name = "MR_VEHICLE_AI_ID_01"
trainer.publish_iteration(
    project.id, iteration.id, publish_iteration_name, PREDICTION_RESOURCE_ID
)
print(f"Model published successfully with name: {publish_iteration_name}")

Starting training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: 

In [91]:
# Create Dictionary of with tags as keys, and lists of randomly choosen image paths as values. The image paths are select from the validation images folder.

import os
import random

# Define your folder path and the number of images to select per tag
test_image_root = "/content/val"
images_per_tag = 100

# Collect all images for each tag
tag_images = {}

for tag_folder in os.listdir(test_image_root):
    folder_path = os.path.join(test_image_root, tag_folder)

    tag = tag_folder.split()[-1]

    # List all images in the folder
    images = [
        os.path.join(folder_path, img)
        for img in os.listdir(folder_path)
        if img.lower().endswith(('jpg', 'jpeg', 'png'))
    ]

    # If the tag already exists, extend the list; otherwise, create a new list
    if tag in tag_images:
        tag_images[tag].extend(images)
    else:
        tag_images[tag] = images

# Limit the total number of images per tag to `images_per_tag`
for tag, images in tag_images.items():
    tag_images[tag] = random.sample(images, min(images_per_tag, len(images)))

In [92]:
# Display Seleted Images as a Dataframe

import pandas as pd

# Assuming tag_images is your dictionary
df = pd.DataFrame.from_dict(tag_images, orient='index').transpose()
df

,wagon,convertible,minivan,cab,Negative,van,truck,coupe,suv,sedan,hatchback
0,/content/val/wagon/07324.jpg,/content/val/convertible/07620.jpg,/content/val/minivan/00006.jpg,/content/val/cab/06303.jpg,/content/val/Negative/h_ec90dc5e.jpg,/content/val/van/00754.jpg,/content/val/truck/A9V4KX9LLLGB.jpg,/content/val/coupe/06108.jpg,/content/val/suv/06102.jpg,/content/val/sedan/00217.jpg,/content/val/hatchback/07111.jpg
1,/content/val/wagon/07653.jpg,/content/val/convertible/01549.jpg,/content/val/minivan/06574.jpg,/content/val/cab/05597.jpg,/content/val/Negative/h_f7abfe07.jpg,/content/val/van/02706.jpg,/content/val/truck/G2AOUPT2N57A.jpg,/content/val/coupe/03477.jpg,/content/val/suv/04075.jpg,/content/val/sedan/06827.jpg,/content/val/hatchback/06915.jpg
2,/content/val/wagon/06132.jpg,/content/val/convertible/04687.jpg,/content/val/minivan/05845.jpg,/content/val/cab/05860.jpg,/content/val/Negative/h_d3964c65.jpg,/content/val/van/07994.jpg,/content/val/truck/5I8NM8MBOCYH.jpg,/content/val/coupe/06945.jpg,/content/val/suv/02809.jpg,/content/val/sedan/03633.jpg,/content/val/hatchback/07686.jpg
3,/content/val/wagon/04139.jpg,/content/val/convertible/06879.jpg,/content/val/minivan/07349.jpg,/content/val/cab/06348.jpg,/content/val/Negative/h_e5df5064.jpg,/content/val/van/06686.jpg,/content/val/truck/W6719ZJ817Q8.jpg,/content/val/coupe/05341.jpg,/content/val/suv/03019.jpg,/content/val/sedan/03667.jpg,/content/val/hatchback/06960.jpg
4,/content/val/wagon/00591.jpg,/content/val/convertible/00500.jpg,/content/val/minivan/00406.jpg,/content/val/cab/01657.jpg,/content/val/Negative/h_f34a3151.jpg,/content/val/van/07134.jpg,/content/val/truck/NZUBW7ZABM0I.jpg,/content/val/coupe/02121.jpg,/content/val/suv/06131.jpg,/content/val/sedan/04024.jpg,/content/val/hatchback/05729.jpg
...,...,...,...,...,...,...,...,...,...,...,...
95,/content/val/wagon/04525.jpg,/content/val/convertible/07431.jpg,/content/val/minivan/03826.jpg,/content/val/cab/06552.jpg,None,/content/val/van/03438.jpg,/content/val/truck/DY3QEEY19DMJ.jpg,/content/val/coupe/03899.jpg,/content/val/suv/06061.jpg,/content/val/sedan/04569.jpg,/content/val/hatchback/00707.jpg
96,/content/val/wagon/05520.jpg,/content/val/convertible/02438.jpg,/content/val/minivan/00286.jpg,/content/val/cab/02063.jpg,None,/content/val/van/05156.jpg,/content/val/truck/XZ705LN30AF3.jpg,/content/val/coupe/02112.jpg,/content/val/suv/04109.jpg,/content/val/sedan/06088.jpg,/content/val/hatchback/07691.jpg
97,/content/val/wagon/03948.jpg,/content/val/convertible/02283.jpg,/content/val/minivan/06092.jpg,/content/val/cab/03973.jpg,None,/content/val/van/04621.jpg,/content/val/truck/LI5VLB09SDOV.jpg,/content/val/coupe/01629.jpg,/content/val/suv/00440.jpg,/content/val/sedan/00477.jpg,/content/val/hatchback/00545.jpg
98,/content/val/wagon/05831.jpg,/content/val/convertible/02813.jpg,/content/val/minivan/05983.jpg,/content/val/cab/06023.jpg,None,/content/val/van/07713.jpg,/content/val/truck/THEV6HDQQ3SI.jpg,/content/val/coupe/06501.jpg,/content/val/suv/00914.jpg,/content/val/sedan/02783.jpg,/content/val/hatchback/00325.jpg


In [ ]:
# Code to Get a Prediction from the Enpoint for a Given Image

import os
import requests
import random
import json
from pathlib import Path

def predict_image(image_path):
    try:
        with open(image_path, "rb") as image_file:
            headers = {
                "Prediction-Key": PREDICTION_KEY,
                "Content-Type": "application/octet-stream"
            }
            url = f"{PREDICTION_ENDPOINT}/customvision/v3.0/Prediction/{project.id}/classify/iterations/{publish_iteration_name}/image"

            response = requests.post(url, headers=headers, data=image_file)

            # Check for a successful status code
            if response.status_code == 200:
                return response.json()
            else:
                # Log or handle non-200 status codes
                print(f"Failed to get a valid response for image {image_path}. Status code: {response.status_code}")
                print(f"Response: {response.text}")
                return None

    except requests.exceptions.RequestException as e:
        # Handle exceptions raised by requests
        print(f"An error occurred while predicting image {image_path}: {e}")
        return None


In [93]:
# Code to Get a Prediction from the Enpoint for a Given Image, and to Compress an Image if it is larger than 4MB limit for the Prediction Endpoint.

import os
import requests
import random
import json
from pathlib import Path
from PIL import Image  # Import the PIL library to handle image compression
import io

def compress_image(image_path, max_size_mb=4, margin=0.95):
    """Compress or resize the image to ensure it is below the specified max size."""
    max_size_bytes = max_size_mb * 1024 * 1024 * margin
    image = Image.open(image_path)

    # Compress and resize until the image is under the max size
    quality = 85  # Starting quality for compression
    while True:
        # Save the image to a bytes buffer
        buffer = io.BytesIO()
        image.save(buffer, format="JPEG", quality=quality)
        size = buffer.tell()

        if size <= max_size_bytes or quality <= 10:  # Stop if size is small enough or quality is too low
            break
        quality -= 5  # Reduce quality incrementally

    buffer.seek(0)  # Reset buffer pointer to the beginning
    return buffer

def predict_image(image_path):
    try:
        # Check the image size and compress if necessary
        image_size = os.path.getsize(image_path)
        if image_size > 4 * 1024 * 1024:  # Check if the image is larger than 4 MB
            print(f"Compressing image {image_path} as it is larger than 4 MB...")
            image_file = compress_image(image_path)
        else:
            image_file = open(image_path, "rb")

        headers = {
            "Prediction-Key": PREDICTION_KEY,
            "Content-Type": "application/octet-stream"
        }
        url = f"{PREDICTION_ENDPOINT}/customvision/v3.0/Prediction/{project.id}/classify/iterations/{publish_iteration_name}/image"

        response = requests.post(url, headers=headers, data=image_file)
        image_file.close()  # Close the file after the request

        # Check for a successful status code
        if response.status_code == 200:
            return response.json()
        else:
            # Log or handle non-200 status codes
            print(f"Failed to get a valid response for image {image_path}. Status code: {response.status_code}")
            print(f"Response: {response.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle exceptions raised by requests
        print(f"An error occurred while predicting image {image_path}: {e}")
        return None

In [95]:
# Test the Model against Validation Data using the Prediction API

import time

# Start the timer
start_time = time.time()

correct_predictions = 0
total_predictions = 0
tag_stats = {tag: {"correct": 0, "total": 0} for tag in tag_images}

# Loop over each tag and its corresponding images
for tag, images in tag_images.items():
    # Select up to 50 random images from the list
    print(f"\nTesting Tag: {tag}\n")
    selected_images = random.sample(images, min(images_per_tag, len(images)))

    correct_predictions = 0
    total_predictions = 0

    # Test each selected image
    for index, image_path in enumerate(selected_images):
        print(f"Testing Image {index + 1} of {len(selected_images)}")
        prediction_result = predict_image(image_path)

        # Check if the prediction_result is None
        if prediction_result is None:
            print(f"Skipping image {image_path} due to failed prediction.")
            continue  # Skip to the next image

        # Get the tag with the highest probability
        try:
            predicted_tag = max(prediction_result["predictions"], key=lambda p: p["probability"])["tagName"]
        except (KeyError, ValueError) as e:
            # Handle cases where "predictions" is missing or empty
            print(f"Error processing prediction for image {image_path}: {e}")
            continue  # Skip to the next image

        # Check if the prediction is correct
        if predicted_tag == tag:
            correct_predictions += 1
            tag_stats[tag]["correct"] += 1
        tag_stats[tag]["total"] += 1
        total_predictions += 1

    # Calculate accuracy safely
    accuracy = tag_stats[tag]["correct"] / tag_stats[tag]["total"] if tag_stats[tag]["total"] > 0 else 0
    print(f"\nAccuracy for Tag {tag}: {accuracy}\n")

# End the timer
end_time = time.time()

# Calculate elapsed time in seconds
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")


Testing Tag: wagon

Testing Image 1 of 100
Testing Image 2 of 100
Testing Image 3 of 100
Testing Image 4 of 100
Testing Image 5 of 100
Testing Image 6 of 100
Testing Image 7 of 100
Testing Image 8 of 100
Testing Image 9 of 100
Testing Image 10 of 100
Testing Image 11 of 100
Testing Image 12 of 100
Testing Image 13 of 100
Testing Image 14 of 100
Testing Image 15 of 100
Testing Image 16 of 100
Testing Image 17 of 100
Testing Image 18 of 100
Testing Image 19 of 100
Testing Image 20 of 100
Testing Image 21 of 100
Testing Image 22 of 100
Testing Image 23 of 100
Compressing image /content/val/wagon/06844.jpg as it is larger than 4 MB...
Testing Image 24 of 100
Testing Image 25 of 100
Testing Image 26 of 100
Testing Image 27 of 100
Testing Image 28 of 100
Testing Image 29 of 100
Testing Image 30 of 100
Testing Image 31 of 100
Testing Image 32 of 100
Testing Image 33 of 100
Testing Image 34 of 100
Testing Image 35 of 100
Testing Image 36 of 100
Testing Image 37 of 100
Testing Image 38 of 100


In [ ]:
# Calculate Overall Accuracy

accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"Overall Accuracy: {accuracy:.2%}")
print("Tag-wise Stats:")
for tag, stats in tag_stats.items():
    tag_accuracy = stats["correct"] / stats["total"] if stats["total"] > 0 else 0
    print(f"{tag}: {tag_accuracy:.2%} ({stats['correct']}/{stats['total']})")

Overall Accuracy: 93.00%
Tag-wise Stats:
Coupe: 59.00% (59/100)
Convertible: 79.00% (79/100)
Sedan: 71.00% (71/100)
Cab: 86.00% (86/100)
Van: 98.00% (98/100)
Hatchback: 70.00% (70/100)
SUV: 81.00% (81/100)
Wagon: 78.00% (78/100)
Minivan: 93.00% (93/100)
